In [9]:
import sys
import os

In [10]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K

In [11]:
K.clear_session()

In [12]:
data_training = './data/entrenamiento'
data_testing = './data/validacion'

In [27]:
epocas = 20
altura, longitud = 150, 150
batch_size = 32
pasos = 1000
pasos_validacion = 300
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3,3)
tamano_filtro2 = (2,2)
tamano_pool = (2,2)
clases = 42
lr = 0.0005

In [28]:
entrenamiento_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True
)

In [29]:
validacion_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [30]:
imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
    data_training,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 5806 images belonging to 42 classes.


In [31]:
imagen_validacion = validacion_datagen.flow_from_directory(
    data_testing,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 84 images belonging to 42 classes.


In [32]:
cnn = Sequential()
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding='same', input_shape = (altura, longitud, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same", activation = 'relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

In [33]:
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

In [34]:
cnn.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr), metrics=['accuracy'])

In [35]:
cnn.fit_generator(
    imagen_entrenamiento,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=imagen_validacion,
    validation_steps=pasos_validacion
)

Epoch 1/20
1000/1000 [==============================] - 1466s 1s/step - loss: 3.5026 - acc: 0.1151 - val_loss: 3.6399 - val_acc: 0.0833
Epoch 2/20
 280/1000 [=======>......................] - ETA: 12:52 - loss: 3.2513 - acc: 0.1489

KeyboardInterrupt: 

In [ ]:
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')